Sprint 9 - Proyecto

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

Tienes acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos (del proyecto del sprint de Análisis estadístico de datos). Para esta tarea de clasificación debes crear un modelo que escoja el plan correcto. Como ya hiciste el paso de procesar los datos, puedes lanzarte directo a crear el modelo.

Desarrolla un modelo con la mayor exactitud posible. En este proyecto, el umbral de exactitud es 0.75. Usa el dataset para comprobar la exactitud.

PASO 1: ABRIR Y EXAMINAR EL ARCHIVO DE DATOS

1.1Carga de librerías

In [42]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


RND = 54321

1.2.Carga de datos y verificar datos ausentes(no tenemos datos ausentes !)

In [43]:
df = pd.read_csv(r"C:\Users\jonat\Desktop\DATA_SCIENTIST\SPRINT_9_Introduccion_al_machine_learning\users_behavior.csv")
df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


PASO 2: SEGMENTAR EN ENTRENAMIENTO / VALIDACION / PRUEBA
PROPUESTA:60 TRAIN, 20 VALID, 20 TEST

2.1.Separar features y target

In [44]:
X = df[['calls','minutes','messages','mb_used']]
y = df['is_ultra']

2.2.Segmentación: 60% train, 20% valid, 20% test

In [45]:
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.20, random_state=RND, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X_temp, y_temp, test_size=0.25, random_state=RND, stratify=y_temp)

In [46]:
print(f"Train: {X_train.shape}, Valid: {X_valid.shape}, Test: {X_test.shape}")
print("Distribución de clases (train/valid/test):",
      y_train.mean().round(3), y_valid.mean().round(3), y_test.mean().round(3))

Train: (1928, 4), Valid: (643, 4), Test: (643, 4)
Distribución de clases (train/valid/test): 0.307 0.306 0.306


PASO 3: INVESTIGAR LA CALIDAD DE DIFERENTES MODELOS CAMBIANDO HIPERPARAMETROS (PROBE CON LOGISTICREGRESSION, DECISIONTREE Y RANDOMFOREST)

3.1 Arbol de decision: variar  con max_depth

In [47]:
for d in [3,5,7,9,None]:
    clf = DecisionTreeClassifier(max_depth=d, random_state=RND)
    clf.fit(X_train, y_train)
    acc = accuracy_score(y_valid, clf.predict(X_valid))
    print(f"DecisionTree max_depth={d} -> Acc(Valid)={acc:.4f}")
    if acc > best_acc:
        best_name, best_model, best_acc = f"DecisionTree(d={d})", clf, acc

DecisionTree max_depth=3 -> Acc(Valid)=0.8087
DecisionTree max_depth=5 -> Acc(Valid)=0.8118
DecisionTree max_depth=7 -> Acc(Valid)=0.8025
DecisionTree max_depth=9 -> Acc(Valid)=0.7947
DecisionTree max_depth=None -> Acc(Valid)=0.7403


3.2 Random Forest: variar n_estimators y max_depth

In [48]:
for n in [100,200,400]:
    for d in [None,5,10]:
        rf = RandomForestClassifier(n_estimators=n, max_depth=d, random_state=RND)
        rf.fit(X_train, y_train)
        acc = accuracy_score(y_valid, rf.predict(X_valid))
        print(f"RandomForest n={n}, depth={d} -> Acc(Valid)={acc:.4f}")
        if acc > best_acc:
            best_name, best_model, best_acc = f"RandomForest(n={n},d={d})", rf, acc

RandomForest n=100, depth=None -> Acc(Valid)=0.8087
RandomForest n=100, depth=5 -> Acc(Valid)=0.8118
RandomForest n=100, depth=10 -> Acc(Valid)=0.8149
RandomForest n=200, depth=None -> Acc(Valid)=0.8087
RandomForest n=200, depth=5 -> Acc(Valid)=0.8134
RandomForest n=200, depth=10 -> Acc(Valid)=0.8149
RandomForest n=400, depth=None -> Acc(Valid)=0.8072
RandomForest n=400, depth=5 -> Acc(Valid)=0.8149
RandomForest n=400, depth=10 -> Acc(Valid)=0.8149


 3.3 Regresión logística: variar solver/C (muy básico)

In [49]:
for C in [0.5,1.0,3.0,10.0]:
    lr = LogisticRegression(max_iter=1000, solver='lbfgs', C=C, random_state=RND)
    lr.fit(X_train, y_train)
    acc = accuracy_score(y_valid, lr.predict(X_valid))
    print(f"LogReg C={C} -> Acc(Valid)={acc:.4f}")
    if acc > best_acc:
        best_name, best_model, best_acc = f"LogReg(C={C})", lr, acc

print(f"\nMejor en validación: {best_name} con Acc={best_acc:.4f}")

LogReg C=0.5 -> Acc(Valid)=0.7558
LogReg C=1.0 -> Acc(Valid)=0.7558
LogReg C=3.0 -> Acc(Valid)=0.7558
LogReg C=10.0 -> Acc(Valid)=0.7558

Mejor en validación: RandomForest con Acc=0.8258


PASO 4: REENTRENAR EL MEJOR MODELO CON TRAIN + VALID Y EVALUAR TEST

In [55]:
X_train_full = pd.concat([X_train, X_valid], axis=0)
y_train_full = pd.concat([y_train, y_valid], axis=0)

final_model = candidates[best_name]['pipeline'].set_params(**best_params)
final_model.fit(X_train_full, y_train_full)

test_acc = accuracy_score(y_test, final_model.predict(X_test))
print(f"\nExactitud en TEST = {test_acc:.4f} (objetivo ≥ 0.75)")


Exactitud en TEST = 0.8149 (objetivo ≥ 0.75)


PASO 5: PRUEBA DE COORDURA

5.1 Baseline rapido, siempre predecir la clase mayoritaria del train_full

In [56]:
majority = int(y_train_full.mean() >= 0.5)
baseline_acc = (y_test.values == majority).mean()
print(f"Baseline mayoría (TEST) = {baseline_acc:.4f}")

Baseline mayoría (TEST) = 0.6936


5.2. Etiquetas barajeadas, 0.5 si es balanceado

In [59]:
y_shuf = y_train_full.sample(frac=1.0, random_state=RND)

shuf_model = candidates[best_name]['pipeline'].set_params(**best_params)
shuf_model.fit(X_train_full, y_shuf)

shuf_acc = accuracy_score(y_test, shuf_model.predict(X_test))
print(f"Con etiquetas barajadas (TEST) = {shuf_acc:.4f}  <-- debería acercarse a ~0.5 si las clases están balanceadas")


Con etiquetas barajadas (TEST) = 0.6967  <-- debería acercarse a ~0.5 si las clases están balanceadas


CONCLUSIONES

Dividi el conjunto en train/valid/test para conservar la proporción de clases: 60%/20%/20%. Esta elección balancea el tamaño suficiente de entrenamiento con un bloque de validación para selección de modelo y un bloque de prueba independiente para evaluación final.

Evaluación y selección de modelos:Compare tres familias, DecisionTreeClassifier, RandomForestClassifier y LogisticRegression, ya que eres un problema de clasifiacino de categoria y no se estimacinoes numericas

Reentrene el modelo ganador con train+valid y se evaluó en test

RandomForest suele ofrecer mejor desempeño en este problema por su capacidad de capturar relaciones no lineales sin requerir escalado.

DecisionTree es interpretable pero sensible a max_depth (riesgo de sobreajuste).

LogisticRegression funciona como un baseline sólido; cuando la frontera es más lineal, puede acercarse al bosque.